In [1]:
import os
import pickle
import PIL.Image
import numpy as np
import pandas as pd
import sys

# path to stylegan encoder locally
sys.path.insert(1, '/home/bobby/APhotoADay/data/stylegan-encoder')

import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

/home/bobby/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bobby/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bobby/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bobby/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

/home/bobby/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bobby/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bobby/miniconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bobby/miniconda3/lib/python3.7/site-packages/tensorboar

In [2]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
def generate_image(latent_vector, size):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((size, size))

def show(latent_vector):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    ax[i].imshow(generate_image(latent_vector))
    ax[i].set_title(latent_vector)
    [x.axis('off') for x in ax]
    plt.show()

In [5]:
# Receive input

real_data_path = input("Enter path for real data: ")
fake_latent_path = input("Enter path for fake latent: ")
start_age = int(input("Enter start age (days): "))
end_age = int(input("Enter end age (days): "))
num_of_vid = input("Enter video number: ")

experiment = "video" + num_of_vid + "_plot_" + str(start_age) + "_to_" + str(end_age)

Enter path for real data: real_data
Enter path for fake latent: fake_latent
Enter start age (days): 6940
Enter end age (days): 8790
Enter video number: 1


In [6]:
# Decode fakes

fake_data_path = input("Enter path for fake data: ")

if not os.path.exists(fake_data_path):
    os.mkdir(fake_data_path)

for file in os.listdir(fake_latent_path):
    img = generate_image(np.load(fake_latent_path + "/" + file), 1024)
    img.save(fake_data_path + "/" + file[:-4] + ".png")

Enter path for fake data: fake_data


In [7]:
# Display GIFs

# def gif_create(delay, path_to_frames, output):
#     os.system("convert -delay " + delay + " -loop 0 " + path_to_frames + "/*.png Results/" + experiment + "/" + output + ".gif")

# gif_create("10", real_data_path, "real")
# gif_create("10", fake_data_path, "fake")




# python3 ../rename3.py . 0

# ffmpeg -f image2 -framerate 25 -pattern_type sequence -start_number 0 -r 2 -i %d.png -vcodec libx264 -acodec aac real_broken.mp4

# ffmpeg -i real_broken.mp4 -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p real.mp4